# YOFO - You Only Fisheye Once

Pipeline for Localize fisheye image using global descriptor

In [1]:
# Check if we have NVIDIA Driver installed !
!nvidia-smi

Wed Mar 13 12:32:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:08:00.0 Off |                  N/A |
| 69%   72C    P2             380W / 420W |   4454MiB / 24576MiB |     97%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Import Stuff

In [5]:
# Imports
# Date Time
from dateutil import tz
from datetime import datetime

# Libraries
import sys
import os
import copy
import json
import numpy as np
import shutil
import PIL

# Scikit-Learn
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets, metrics

# PYTORCH !
import torch
import torchinfo
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# Visualization
%matplotlib widget
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
from torch.utils.tensorboard import SummaryWriter

# Path for Import Internal Modules
sys.path.append("/workspace/fisheye-vpr")


### Fix Seed for Regeneration

In [8]:
## Fix Seed for Regeneration
torch.manual_seed(777)

---

### Import Model

In [12]:
# Main Model
from models.vpr_model import VPRModel

# Feature Extractor
from models.encoders.resnet18 import Resnet18Encoder
from models.encoders.vgg16 import VGG16Encoder

# Clustering
from models.clustering.netvlad import NetVLAD

# Loss
from models.loss_function.HardTripletLoss import HardTripletLoss

### Dataset Selection

In [13]:
from dataloaders.isl2_3places_fisheye import ISL2_3Places_Fisheye_Dataset
dataset = ISL2_3Places_Fisheye_Dataset()

---

### Model Assembling

In [15]:
netvlad_config = {
    "num_clusters": 20,
    "desc_dim": 512,            # Up to the Feature Extraction Module
    "alpha": 100.0,
    "normalize_input": True,
}
model = VPRModel(
    feature_extractor=Resnet18Encoder(),
    clustering=NetVLAD(**netvlad_config)
)

[RESNET18] Output Dim Size: 512
NetVLAD Module initialized !
 - self.num_clusters : 20
 - self.dim : 512
 - self.alpha : 100.0
 - self.normalize_input : True


---